In [162]:
import os
import cv2
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Conv2D,Dense,MaxPool2D,Dropout,Flatten
from keras.models import Sequential
from skimage import io
from tensorflow.keras.utils import to_categorical
from matplotlib.image import imread

In [163]:
# Set Path
path = os.path.expanduser('~') + '/inhamath/kaggle_FSWJ'

# train & labels
train = [io.imread('{0}/train/{1}.Bmp'.format(path,str(i)),as_gray=True) for i in range(1,6283)]
labels = pd.read_csv('{0}/trainLabels.csv'.format(path))

In [164]:
mapp={}
a='abcdefghijklmnopqrstuvwxyz'
count=0
for x in range(10):
    mapp[x]=count
    count+=1
for y in a:
    mapp[count]=y.upper()
    count+=1
for y in a:
    mapp[count]=y
    count+=1

In [165]:
train_list = [cv2.resize(image,(28,28)) for image in train]
features_for_conv=np.array(train_list)
features_for_conv=features_for_conv[:,:,:,np.newaxis]

In [166]:
# 모델 
cmodel=Sequential()
cmodel.add(Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1),data_format='channels_last'))
cmodel.add(MaxPool2D((2,2)))
cmodel.add(Dropout(0.25))
cmodel.add(Conv2D(64,(3,3),activation='relu'))
cmodel.add(MaxPool2D((2,2)))
cmodel.add(Dropout(0.25))
cmodel.add(Flatten())
cmodel.add(Dense(128,activation='relu'))
cmodel.add(Dense(62,activation='softmax'))

In [167]:
Labels=pd.get_dummies(labels['Class'])

In [168]:
cmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [169]:
cmodel.fit(features_for_conv,Labels.iloc[:-1],epochs=30)

Epoch 1/30
197/197 [==============================] - 1s 3ms/step - loss: 3.9756 - accuracy: 0.0630
Epoch 2/30
197/197 [==============================] - 1s 3ms/step - loss: 3.6974 - accuracy: 0.1468
Epoch 3/30
197/197 [==============================] - 1s 3ms/step - loss: 2.8633 - accuracy: 0.3443
Epoch 4/30
197/197 [==============================] - 1s 3ms/step - loss: 2.1134 - accuracy: 0.4819
Epoch 5/30
197/197 [==============================] - 1s 3ms/step - loss: 1.6749 - accuracy: 0.5673
Epoch 6/30
197/197 [==============================] - 1s 3ms/step - loss: 1.4504 - accuracy: 0.6054
Epoch 7/30
197/197 [==============================] - 1s 3ms/step - loss: 1.2704 - accuracy: 0.6531
Epoch 8/30
197/197 [==============================] - 1s 3ms/step - loss: 1.1416 - accuracy: 0.6850
Epoch 9/30
197/197 [==============================] - 1s 3ms/step - loss: 1.0564 - accuracy: 0.7041
Epoch 10/30
197/197 [==============================] - 1s 3ms/step - loss: 0.9746 - accuracy: 0.7246

In [170]:
lab=[]
test=[]

for i in os.listdir(path + '/test/'):
    test.append(io.imread(path + '/test/'+i,as_gray=True))
    lab.append(i.split('.')[0])

In [171]:
test_img=np.array([cv2.resize(image,(28,28)) for image in test])
test_img=test_img[:,:,:,np.newaxis]

In [172]:
predictions=cmodel.predict(test_img)

In [173]:
predictions=np.argmax(predictions,axis=1)

In [174]:
lit=[]
for x in predictions:
    lit.append(mapp.get(x))
lit

['T',
 'a',
 'L',
 'P',
 'S',
 'I',
 'E',
 'c',
 'r',
 'e',
 'O',
 'C',
 'E',
 'O',
 'P',
 'E',
 'X',
 'O',
 'R',
 'C',
 'E',
 'D',
 'D',
 'O',
 'j',
 'R',
 'C',
 'T',
 'e',
 'C',
 'G',
 'E',
 'r',
 'N',
 'T',
 'i',
 'u',
 'N',
 'P',
 'e',
 'N',
 'A',
 'i',
 'i',
 'L',
 'M',
 's',
 'I',
 's',
 'A',
 'T',
 'V',
 'N',
 'e',
 'H',
 'T',
 'G',
 'O',
 'O',
 'L',
 'S',
 'A',
 'q',
 'A',
 'r',
 'e',
 'N',
 'V',
 'd',
 'T',
 'o',
 'E',
 'L',
 'I',
 'T',
 'N',
 'E',
 'e',
 'P',
 'N',
 'N',
 'N',
 'R',
 'I',
 'i',
 'A',
 'L',
 'M',
 'u',
 'E',
 'R',
 'C',
 'R',
 'S',
 'D',
 'C',
 'T',
 'i',
 'N',
 'N',
 'O',
 'O',
 'I',
 9,
 'n',
 's',
 8,
 2,
 'N',
 'E',
 'L',
 2,
 'E',
 'H',
 'O',
 'M',
 'V',
 't',
 'U',
 'C',
 'A',
 'y',
 'O',
 'V',
 'N',
 'q',
 'T',
 'O',
 1,
 'r',
 'I',
 'E',
 'I',
 'R',
 5,
 'V',
 'i',
 'y',
 'E',
 'S',
 'D',
 'R',
 'E',
 'O',
 'R',
 'p',
 'N',
 'H',
 'F',
 'a',
 'A',
 'I',
 'n',
 'H',
 'I',
 'n',
 'r',
 'R',
 'P',
 'T',
 'I',
 'O',
 't',
 8,
 'n',
 4,
 3,
 'T',
 'O',
 'E'

In [175]:
sub=pd.DataFrame({'ID':lab,'Class':lit})
sub.to_csv('julia_yeryung.csv',index=False)